<a href="https://colab.research.google.com/github/cat-Lsb-20206394/HisTour-AI/blob/main/data_to_csv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests
!pip install lxml

In [48]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

# API Endpoint (Base URL)
base_url_list = "http://www.khs.go.kr/cha/SearchKindOpenapiList.do"
base_url_detail = "http://www.khs.go.kr/cha/SearchKindOpenapiDt.do"

kind_codes = ["11","12","13","14","15","16","17","18","21","22","23","24","25"]
city_codes = ["11","21","22","23","24","25","26","45","31","32","33","34","35","36","37","38","50"]

all_data = []

for kdcd in kind_codes:
  for ctcd in city_codes:
    page_idx = 1
    while True:
      params = {
        "ccbaKdcd": kdcd,
        "ccbaCtcd": ctcd,
        "pageUnit": 100,
        "pageIndex": page_idx
      }
      response = requests.get(base_url_list, params=params)
      if response.status_code != 200:
        print(f"목록 API 에러 code: {response.status_code}")
        break

      root = ET.fromstring(response.text)
      items = root.findall(".//item")

      if not items:
        break  # 더 이상 데이터 없으면 종료

      for item in items:
        # 목록 API에서 기본 정보 추출
        name = item.findtext("ccbaMnm1")
        kind = item.findtext("ccmaName")
        location = item.findtext("ccbaCtcdNm", "") + " " + item.findtext("ccsiName", "")
        lon = item.findtext("longitude")
        lat = item.findtext("latitude")

        # 상세조회에 필요한 파라미터(관리번호, 종목코드, 시도코드)
        asno = item.findtext("ccbaAsno")
        kdcd_val = item.findtext("ccbaKdcd")  # 혹은 kdcd
        ctcd_val = item.findtext("ccbaCtcd")  # 혹은 ctcd

        # 상세 정보 API 호출
        detail_params = {
          "ccbaAsno": asno,
          "ccbaKdcd": kdcd_val,
          "ccbaCtcd": ctcd_val
        }
        detail_res = requests.get(base_url_detail, params=detail_params)
        detail_content = ""

        if detail_res.status_code == 200:
          detail_root = ET.fromstring(detail_res.text)
          detail_item = detail_root.find(".//item")
          if detail_item is not None:
            detail_content = detail_item.findtext("content", "")
            detail_address = detail_item.findtext("ccbaLcad", "")
        else:
          print(f"상세 API 에러 code: {detail_res.status_code}, asno={asno}")

        # 최종 레코드
        all_data.append({
          # "종목코드": kdcd_val,
          # "시도코드": ctcd_val,
          # "관리번호": asno,
          "문화재명": name,
          "분류":kind,
          "소재지": location.strip(),
          "상세주소": detail_address,
          "위도": lon,
          "경도": lat,
          "상세설명": detail_content
        })

      page_idx += 1


print(f"{len(all_data)}")

df = pd.DataFrame(all_data)
df.head()


KeyboardInterrupt: 

In [42]:
df["위도"] = pd.to_numeric(df["위도"], errors="coerce")
df["경도"] = pd.to_numeric(df["경도"], errors="coerce")

df_not_zero = df[
    (df["위도"] != 0) &
    (df["경도"] != 0) &
    df["위도"].notna() &
    df["경도"].notna()
]

print(df_not_zero.shape)
df_not_zero.head()

(1157, 7)


,문화재명,분류,소재지,상세주소,위도,경도,상세설명
0,서울 숭례문,국보,서울 중구,\n\t\t\t\n\t \n\t ...,126.975313,37.559975,조선시대 한양도성의 정문으로 남쪽에 있다고 해서 남대문이라고도 불렀다. 현재 서울에...
1,서울 원각사지 십층석탑,국보,서울 종로구,\n\t\t\t\n\t \n\t ...,126.988208,37.571546,"원각사는 지금의 탑골공원 자리에 있었던 절로, 조선 세조 11년(1465)에 세웠다..."
2,서울 북한산 신라 진흥왕 순수비,국보,서울 용산구,\n\t\t\t\n\t \n\t ...,126.980350,37.524041,"신라 진흥왕(재위 540∼576)이 세운 순수척경비(巡狩拓境碑) 가운데 하나로, 한..."
3,청자 사자형뚜껑 향로,국보,서울 용산구,\n\t\t\t\n\t \n\t ...,126.980350,37.524041,"고려 청자의 전성기인 12세기경에 만들어진 청자향로로, 높이 21.2㎝, 지름 16..."
4,청자 어룡형 주전자,국보,서울 용산구,\n\t\t\t\n\t \n\t ...,126.980350,37.524041,"고려 청자의 전성기인 12세기경에 만들어진 청자주전자로 높이 24.4cm, 밑지름 ..."


In [43]:
df_not_zero.to_csv("seoul_heritage.csv", index=False, encoding="utf-8-sig")